<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

В рамках данного проекта необходимо защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. И обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from numpy import linalg as LA

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
print (df.head(20))
df.info()
df.shape

    Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0     1     41.0   49600.0            1                  0
1     0     46.0   38000.0            1                  1
2     0     29.0   21000.0            0                  0
3     0     21.0   41700.0            2                  0
4     1     28.0   26100.0            0                  0
5     1     43.0   41000.0            2                  1
6     1     39.0   39700.0            2                  0
7     1     25.0   38600.0            4                  0
8     1     36.0   49700.0            1                  0
9     1     32.0   51700.0            1                  0
10    1     25.0   36600.0            1                  0
11    1     38.0   29300.0            0                  0
12    0     23.0   39500.0            3                  0
13    0     21.0   55000.0            0                  0
14    0     40.0   43700.0            1                  0
15    1     34.0   23300.0            0                 

(5000, 5)

## Умножение матриц

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
.....
$$

$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' =  P^{-1}(X^T X)^{-1}P^{-T}  P^TX^T  y
$$
$$
w = (X^T X)^{-1} X^T y
$$
$$
w' =  P^{-1}P^{-T} P^T w
$$
Сокращаем P
$$
w' = P^{-1} w
$$

**Ответ:** необходимо признаки умножить на обратимую матрицу, чтобы зашифровать данные

**Обоснование:** при умножении на обратимую матрицу R2 до преобразования должен быть равен R2 после преобразования.

In [3]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']


class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        answer = (test_features @ self.w) + self.w0
        return answer
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


## Алгоритм преобразования

**Алгоритм**

Создаем обратимую матрицу

In [8]:
matrix3 = np.random.normal(10, size=(4,4))
print (matrix3)

[[10.88819758 11.2326134  10.97659872  9.63588639]
 [10.22308819 10.00336607 10.64290974 10.88435868]
 [ 8.1123552   8.18237321 10.500499    9.3542681 ]
 [ 8.18432862 11.1852767   8.97193319 11.43076578]]


**Проверяю матрицу на обратимость**

In [14]:
matrix3_inv = LA.inv(matrix3)
print (matrix3_inv)

[[ 3.93017070e-04  6.70775573e-01 -4.46387525e-01 -2.73745476e-01]
 [ 3.50159038e-01 -5.78348565e-01  6.07651784e-03  2.50553567e-01]
 [ 1.07475070e-01 -4.78273727e-01  5.18440755e-01 -5.94490870e-02]
 [-4.27276838e-01  4.61052307e-01 -9.32571951e-02  8.49711396e-02]]


**Проверяю результат обратимости матрицы matrix3_inv, используя свойство обратной матрицы**\
np.dot(matrix3, matrix3_inv) = np.dot(matrix3_inv, matrix3) = np.eye(matrix3.shape[0])

In [15]:
np.dot(matrix3, matrix3_inv)

array([[ 1.00000000e+00, -1.13918981e-15,  4.50918001e-16,
         2.48986675e-16],
       [ 2.34874400e-16,  1.00000000e+00, -1.08999736e-16,
         1.15613187e-17],
       [-6.79442252e-16,  5.25421084e-17,  1.00000000e+00,
        -1.57989955e-16],
       [-1.39205728e-16, -3.67096342e-17,  4.40115757e-16,
         1.00000000e+00]])

In [18]:
np.allclose(np.dot(matrix3, matrix3_inv), np.eye(matrix3.shape[0]))

True

In [19]:
np.allclose(np.dot(matrix3_inv, matrix3), np.eye(matrix3.shape[0]))

True

**Умножаю признаки на обратимую матрицу для зашифровки данных**

In [20]:
features_m = features @ matrix3
print (features_m)
features_m.shape

                 0             1             2             3
0    -4.521367e+06 -4.167556e+06 -7.250442e+06 -6.171941e+06
1    -3.462918e+06 -3.190859e+06 -5.553561e+06 -4.726536e+06
2    -1.913708e+06 -1.763110e+06 -3.069033e+06 -2.611809e+06
3    -3.801711e+06 -3.505186e+06 -6.096267e+06 -5.190231e+06
4    -2.378942e+06 -2.192318e+06 -3.814933e+06 -3.247087e+06
...            ...           ...           ...           ...
4995 -3.253904e+06 -2.999361e+06 -5.218173e+06 -4.441986e+06
4996 -4.777330e+06 -4.404186e+06 -7.660602e+06 -6.521692e+06
4997 -3.090245e+06 -2.848987e+06 -4.955556e+06 -4.218843e+06
4998 -2.980156e+06 -2.747258e+06 -4.779360e+06 -4.068588e+06
4999 -3.701245e+06 -3.412016e+06 -5.935188e+06 -5.052669e+06

[5000 rows x 4 columns]


(5000, 4)

**Обоснование**

Умножаем признаки на обратимую матрицу

## Проверка алгоритма

In [21]:
features = features_m
target = df['Страховые выплаты']


class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        answer = (test_features @ self.w) + self.w0
        return answer
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249454984507788


## Вывод

После преобразования признаков R2 модели остался такой же как и до преобразования. Модель работает, а признаки зашифрованы.